# Домашнее задание - линейная регрессия

### Работа с признаками (8 баллов)

Скачайте датасет из материалов к уроку или по ссылке https://raw.githubusercontent.com/jupiterzhuo/travel-insurance/master/travel%20insurance.csv


Описание признаков:

* Agency — название страхового агентства
* Agency Type — тип страхового агентства
* Distribution Channel — канал продвижения страхового агентства
* Product Name — название страхового продукта
* Duration — длительность поездки (количество дней)
* Destination — направление поездки
* Net Sales — сумма продаж
* Commission (in value) — комиссия страхового агентства
* Gender — пол застрахованного
* Age — возраст застрахованного

Ответ:
* Claim — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

Обработайте пропущенные значения и примените написанные функции onehot_encode() и minmax_scale().

**Подсказка**: маску для категориальных признаков можно сделать фильтром cat_features_mask = (df.dtypes == "object").values

Подробный анализ и подготовка датасета часто помогают улучшить качество модели. Ниже представлено несколько идей преобразований. Вы можете применить одно или несколько из этих преобразований (а можете не применять), чтобы помочь будущей модели.

1. Посмотрите на количественные признаки. Возможно, в некоторых признаках есть выбросы - значения, которые сильно выбиваются. Такие значения полезно удалять. Советуем присмотреться к колонке Duration)

2. Можно заметить, что one hot encoding сильно раздувает количество столбцов. Радикальное решение - можно попробовать выбросить все категориальные признаки из датасета.

3. Если все-таки оставляете категориальные признаки, то подумайте, как уменьшить количество столбцов после one hot encoding. Признаки с большим количеством значений (Duration - 149! разных стран) можно удалить или попробовать сгруппировать некоторые значения.

4. Downsampling. Датасет достаточно большой, разница в классах огромная. Можно уменьшить число наблюдений с частым ответом.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("travel insurance.csv")
data = data.drop(["Agency", "Product Name", "Destination"], axis=1)
data.Claim.value_counts()

,count
Claim,
No,62399
Yes,927


#### Downsampling. Так как у нас невероятно огромная разница в классах (927 к 62399), то данная часть наиболее важная в предобработке данных


In [ ]:
data = pd.concat([data[data['Claim'] == 'No'].head(2500), data[data['Claim'] == 'Yes']])
data.Claim.value_counts()

,count
Claim,
No,2500
Yes,927


#### Убираем пропуски


In [ ]:
print(data.isna().any()) # -> Gender: True
# Заполним пропуски Gender чаще встречающимся полом
print(data["Gender"].value_counts())
data["Gender"] = data["Gender"].fillna('F')

Agency Type             False
Distribution Channel    False
Claim                   False
Duration                False
Net Sales               False
Commision (in value)    False
Gender                   True
Age                     False
dtype: bool
Gender
M    582
F    568
Name: count, dtype: int64


#### Преобразовываем категориальные значения

In [ ]:
def onehot_encoding(array: np.ndarray) -> np.ndarray:
    features_count = len(set(array))
    objects_count = array.size
    one_hot = np.zeros(features_count * objects_count).reshape(objects_count, features_count)
    array_sorted = list(set(array))
    array_sorted.sort()
    features = {array_sorted[i]: i for i in range(features_count)}
    for i in range(objects_count):
        one_hot[i, features[array[i]]] = 1
    return one_hot.astype(int)

In [ ]:
def minmax_scale(X: np.ndarray) -> np.ndarray:
    X_scaled = []
    for x in X:
        x_scaled = []
        for i in range(x.size):
            znam = X[:, i].max(axis=0) - X[:, i].min(axis=0)
            if znam != 0:
                x_scaled.append((x[i] - X[:, i].min(axis=0)) / znam)
            else:
                x_scaled.append(0.)
        X_scaled.append(x_scaled)

    return np.array(X_scaled)

In [ ]:
# Преобразовываем Agency Type

# print(data["Agency Type"].unique()) # -> ['Travel Agency' 'Airlines']
agency_column = np.array(data["Agency Type"])
data = data.drop("Agency Type", axis=1)
encoded_agency_column = onehot_encoding(agency_column)
data['Travel Agency'] = encoded_agency_column[:, 0]
data['Airlines'] = encoded_agency_column[:, 1]

In [ ]:
# Преобразовываем Distribution Channel

# print(data["Distribution Channel"].unique()) # -> ['Offline' 'Online']
channel_column = np.array(data["Distribution Channel"])
data = data.drop("Distribution Channel", axis=1)
encoded_channel_column = onehot_encoding(channel_column)
data["Online"] = encoded_channel_column[:, 1]

In [ ]:
# Преобразовываем Gender

# print(data["Gender"].unique()) # -> ['F' 'M']
channel_column = np.array(data["Gender"])
data = data.drop("Gender", axis=1)
encoded_channel_column = onehot_encoding(channel_column)
data["Male"] = encoded_channel_column[:, 1]

In [ ]:
# Удаляем выбросы
data["Duration"].describe()
data = data[data["Duration"] <= 200]

In [ ]:
data["Claim"] = np.where(data['Claim'] == 'Yes', 1,0)
y = np.array(data["Claim"])
data.drop(["Claim"], axis=1, inplace=True)

In [ ]:
data_array = np.array(data)
for i in range(data.shape[1]):
  data.iloc[:, [i]] = minmax_scale(data_array[:, i].reshape(len(data_array), 1))

<ipython-input-695-04d6133b9aef>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.93939394 0.93939394 0.32828283 ... 0.03535354 0.13636364 0.06060606]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, [i]] = minmax_scale(data_array[:, i].reshape(len(data_array), 1))
<ipython-input-695-04d6133b9aef>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.64423077 0.54807692 0.17307692 ... 0.15384615 0.15384615 0.13461538]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, [i]] = minmax_scale(data_array[:, i].reshape(len(data_array), 1))


### Применение линейной регрессии (10 баллов)

Это задача классификации, но её можно решить с помощью линейной регрессии, если округлять предсказанный ответ до целого и выбирать ближайший по значению ответ из множества {0, 1}.

Вынесите признак 'Claim' в вектор ответов и разделите датасет на обучающую и тестовую выборку в соотношении 80 к 20. Зафиксируйте random_state.

**Подсказка:** быстро перевести Yes/No в 1/0 можно так - np.where(df['Claim'] == 'Yes', 1,0)

In [ ]:
# разделение на test/train
X_train, X_test, Y_train, Y_test = train_test_split(data, y, train_size=0.75)

Найдите аналитическое решение для обучающей выборки: обычное и регуляризацией l2.

In [ ]:
X_train_analitical = X_train.copy()
X_train_analitical.insert(0, "Zero weight", [1] * len(X_train_analitical))
X_train_analitical
X_test_analitical = X_test.copy()
X_test_analitical.insert(0, "Zero weight", [1] * len(X_test_analitical))

In [ ]:
# посчитайте аналитическое решение
w_analitical = np.array(np.linalg.inv(X_train_analitical.T @ X_train_analitical) @ X_train_analitical.T @ Y_train)
w_analitical

array([ 1.25673486, -0.08553943,  0.9819763 ,  0.149515  , -0.47301887,
        0.39427153, -0.06711942, -0.36494835, -0.04390461])

In [ ]:
# посчитать аналитическое решение с регуляризацией
u = 2
w_analitical_l2 = np.array(np.linalg.inv(X_train_analitical.T @ X_train_analitical - u**2 * np.eye(X_train_analitical.shape[1])) @ X_train_analitical.T @ Y_train)
w_analitical_l2

array([ 0.91811238, -0.08666432, -1.8250501 ,  1.69412497, -0.65957088,
        0.63099053,  0.28712185, -0.59758985, -0.05039857])

In [ ]:
Y_pred_anilitical = []
Y_pred_anilitical_l2 = []
for i in range(X_test_analitical.shape[0]):
  y_pred = y_l2_pred = 0
  for j in range(X_test_analitical.shape[1]):
    y_pred += w_analitical[j] * X_test_analitical.iloc[i, j]
    y_l2_pred += w_analitical_l2[j] * X_test_analitical.iloc[i, j]
  Y_pred_anilitical.append(round(y_pred))
  Y_pred_anilitical_l2.append(round(y_l2_pred))

Постройте модель LinearRegression, примените к тестовой выборке и посчитайте MSE (можно использовать библиотеку sklearn)

In [ ]:
# обучите модель линейной регрессии LinearRegression на обучающей выборке, примените к тестовой
model = LinearRegression()
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred = Y_pred.round()
model.coef_

array([-0.11823274,  1.29843381,  0.35286774, -0.45542851,  0.16921538,
       -0.16921538, -0.36494835, -0.04390461])

In [ ]:
mse_sklearn = mean_squared_error(Y_test, Y_pred)
mse_analitical = mean_squared_error(Y_test, Y_pred_anilitical)
mse_analitical_l2 = mean_squared_error(Y_test, Y_pred_anilitical_l2)

accuracy_sklearn = accuracy_score(Y_test, Y_pred)
accuracy_analitical = accuracy_score(Y_test, Y_pred_anilitical)
accuracy_analitical_l2 = accuracy_score(Y_test, Y_pred_anilitical_l2)
print(mse_sklearn, mse_analitical, mse_analitical_l2)


0.21611253196930946 0.7762148337595908 0.24168797953964194


### Вывод (1 балла)

Напишите краткий вывод по заданию (достаточно пары предложений). Расскажите, какие способы предобработки данных вы выбрали и почему. Насколько хороша ваша модель?

САМОЕ ВАЖНОЕ - использовал андер сэмплинг (необходим, т.к. огромная разница в классах). Избавился от выбросов в столбце Duration и преобразовал столбцы Agency Type, Gender, Distributin Channel, содержащие категориальные признаки, также заполнил пропуски в столбце Dender чаще встречающимся значением этого столбца. Модель для линейной регрессии выдаёт нормальные результаты, MSE\~0.3 (для решения sklearn и аналитического с регуляризацией). Худшие результаты показало аналитическое решение без регуляризации



